In [1]:
# Define your API key
api_key = "7XKSINP2NOC16NzhK53KWf91JQQo2ndDWYxbFmxO944T95iW4LtGALWGbWPM" #fre one
api_key = "DAp9iyzcrjk9dXPcA29NGXtrIub6NUixrw7heI7s6ETd7nSp5Ai0TIr6SOaU"  #paying one
api_key = "4IlixFg8NraryF6hw7x2fl9daC9rdyB3enyWtXjO0TfLBOBrggYceNQcgXr9"  #paying 

api_key = "jEev3izPThtfGDQCyZk4lKbpo3Oc2zp40NEQIZm1ws9x0rrKAeHnsbHJkfzY"



In [ ]:
https://soccer.sportmonks.com/api/v2.0/fixtures/19146831?api_token=7XKSINP2NOC16NzhK53KWf91JQQo2ndDWYxbFmxO944T95iW4LtGALWGbWPM&include=lineup.player,bench.player
https://soccer.sportmonks.com/api/v2.0/fixtures/19146831?api_token=7XKSINP2NOC16NzhK53KWf91JQQo2ndDWYxbFmxO944T95iW4LtGALWGbWPM&include=lineups.detailedposition,bench.player
#Per a trobar posicions exactes podem fer merge amb fifa stats
#Dades de les alineacions d'un partit en concret

recompute the code taking into account v3!!


prompt: 
You are an expert in sportmonks API in using python. We have made the following extraction:
base_url = "https://api.sportmonks.com/v3/football"
endpoint = "/fixtures/19146831"
include = "lineups.player;lineups.detailedposition"
endpoint_url = f"{base_url}{endpoint}?include={include}"
Given this player lineups football data extraction from the match "19146831" , compute a code which saves the from the data outputed, a the match id, each team id (as local_team_id and away_team_id), and the lineup and bench each one as a dictionary which just saves the player_name, player_id position_id, detailed_position_id, height, weight and date of birth. This dictionary will be called local_lineup, away_lineup, local_bench or away_bench.



In [39]:
import requests
import pandas as pd

def extract_match_data(match_list):
    base_url = "https://api.sportmonks.com/v3/football"
    include = "lineups.player;lineups.detailedposition"

    # Initialize an empty list to store the data for all matches
    all_matches_data = []

    for match_id in match_list:
        # Construct the endpoint URL for each match ID
        endpoint_url = f"{base_url}/fixtures/{match_id}?include={include}&api_token={api_key}"

        response = requests.get(endpoint_url)
        data = response.json()

        # Extract match ID and team IDs
        lineups = data['data']['lineups']

        local_team_id = lineups[0]['team_id']
        away_team_id = lineups[-1]['team_id']  # Data from the bottom is the away's team one 
        assert local_team_id != away_team_id, "Error: Local or away team not detected"

        # Initialize lineups and benches
        local_lineup = []
        away_lineup = []
        local_bench = []
        away_bench = []

        # Process each player in lineups
        for lineup in lineups:
            player_info = lineup['player']
            player_dict = {
                'player_name': player_info['name'],
                'player_id': player_info['id'],
                'position_id': player_info['position_id'],
                'detailed_position_id': player_info['detailed_position_id'],
                'height': player_info['height'],
                'weight': player_info['weight'],
                'date_of_birth': player_info['date_of_birth']
            }

            if lineup['type_id'] == 11:  # Initial lineup
                if lineup['team_id'] == local_team_id:
                    local_lineup.append(player_dict)
                else:
                    away_lineup.append(player_dict)
            elif lineup['type_id'] == 12:  # Bench
                if lineup['team_id'] == local_team_id:
                    local_bench.append(player_dict)
                else:
                    away_bench.append(player_dict)

        # Append match data to the list
        match_data = {
            'match_id': match_id,
            'local_team_id': local_team_id,
            'away_team_id': away_team_id,
            'local_lineup': local_lineup,
            'away_lineup': away_lineup,
            'local_bench': local_bench,
            'away_bench': away_bench
        }
        all_matches_data.append(match_data)

    # Create a DataFrame from the list of matches data
    df = pd.DataFrame(all_matches_data)

    return df

match_list = ['19146831', '18841584', '18841576']  # Canviar per llistat de partits a analitzar
df_lineup = extract_match_data(match_list)
df_lineup.head()


,match_id,local_team_id,away_team_id,local_lineup,away_lineup,local_bench,away_bench
0,19146831,66,309,"[{'player_name': 'Warren O'Hora', 'player_id':...","[{'player_name': 'Archie Mair', 'player_id': 3...","[{'player_name': 'Josef Bursik', 'player_id': ...","[{'player_name': 'Davor Zdravkovski', 'player_..."
1,18841584,2356,2650,"[{'player_name': 'Patrik Ulf Anders Carlgren',...","[{'player_name': 'David Jensen', 'player_id': ...","[{'player_name': 'Wessel Dammers', 'player_id'...","[{'player_name': 'Lucas Lissens', 'player_id':..."
2,18841576,2905,7466,"[{'player_name': 'Felix Beijmo', 'player_id': ...","[{'player_name': 'Nathan Trott', 'player_id': ...","[{'player_name': 'Jesper Hansen', 'player_id':...","[{'player_name': 'Kristian Kirkegaard', 'playe..."


In [45]:
for x in df_lineup["local_lineup"][1]:
    print(x["player_name"])

Patrik Ulf Anders Carlgren
Daniel Høegh
Hugo Andersson
John Björkengren
Mikkel Kallesøe
Mads Enggård
Simen Bolkan Nordli
Lasso Coulibaly
Oliver Zandén
Mohammed Gadafi Fuseini
Stephen Odey


# Download data

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Define base URL and API key
BASE_URL = "https://api.sportmonks.com/v3/football/fixtures"
# API_KEY = "4IlixFg8NraryF6hw7x2fl9daC9rdyB3enyWtXjO0TfLBOBrggYceNQcgXr9"  #paying one

API_KEY = "jEev3izPThtfGDQCyZk4lKbpo3Oc2zp40NEQIZm1ws9x0rrKAeHnsbHJkfzY"  #UAB one
# Function to generate date intervals
def generate_date_intervals(start_date, end_date, delta_days):
    intervals = []
    current_date = start_date
    while current_date < end_date:
        next_date = current_date + timedelta(days=delta_days)
        if next_date > end_date:
            next_date = end_date
        intervals.append((current_date.strftime('%Y-%m-%d'), next_date.strftime('%Y-%m-%d')))
        current_date = next_date + timedelta(days=1)
    return intervals

# Generate 100-day intervals
start_date = datetime(2018, 7, 1)
end_date = datetime(2025, 5, 20)
date_intervals = generate_date_intervals(start_date, end_date, 100)


In [2]:
date_intervals

[('2018-07-01', '2018-10-09'),
 ('2018-10-10', '2019-01-18'),
 ('2019-01-19', '2019-04-29'),
 ('2019-04-30', '2019-08-08'),
 ('2019-08-09', '2019-11-17'),
 ('2019-11-18', '2020-02-26'),
 ('2020-02-27', '2020-06-06'),
 ('2020-06-07', '2020-09-15'),
 ('2020-09-16', '2020-12-25'),
 ('2020-12-26', '2021-04-05'),
 ('2021-04-06', '2021-07-15'),
 ('2021-07-16', '2021-10-24'),
 ('2021-10-25', '2022-02-02'),
 ('2022-02-03', '2022-05-14'),
 ('2022-05-15', '2022-08-23'),
 ('2022-08-24', '2022-12-02'),
 ('2022-12-03', '2023-03-13'),
 ('2023-03-14', '2023-06-22'),
 ('2023-06-23', '2023-10-01'),
 ('2023-10-02', '2024-01-10'),
 ('2024-01-11', '2024-04-20'),
 ('2024-04-21', '2024-07-30'),
 ('2024-07-31', '2024-11-08'),
 ('2024-11-09', '2025-02-17'),
 ('2025-02-18', '2025-05-20')]

In [3]:
import requests
import pandas as pd
import os
import time


# Season list (IDs of different leagues)

# season_list = [18378, 19734, 21646,  # Premier League
#                18444, 19744, 21795,  # Bundesliga
#                18441, 19745, 21779,  # Ligue 1
#                18576, 19806, 21818,  # Serie A
#                18462, 19799, 21694,  # La Liga
#                18529, 19896, 21825,  # Primeira Liga
#                18350, 19726, 21730]  # Eredivisie

#-------------18-19,  19-20, 20-21, 21-22, 22-23, 23-24, 24-25
season_list = [12962, 16036, 17420, 18378, 19734, 21646, 23614, # Premier League
               13005, 16264, 17361, 18444, 19744, 21795, 23744,  # Bundesliga
               12935, 16043, 17160, 18441, 19745, 21779, 23643, # Ligue 1
               13158, 16415, 17488, 18576, 19806, 21818, 23746,  # Serie A
               13133, 16326, 17480, 18462, 19799, 21694, 23621,  # La Liga
               13049, 16346, 17463, 18529, 19896, 21825, 23793,   # Primeira Liga
               12946, 16046, 17228, 18350, 19726, 21730, 23628,  # Eredivisie

               12951, 16276, 17138, 18348, 19785, 21693, 23589,  #Pro League (Belgium)
               13251, 16416, 17533, 18710, 20080, 21943, 23839,  #Serie B
               12921, 16010, 17287, 18410, 19692, 21650, 23578,  #Ekstraklasa (Poland) 
               13150, 16329, 17483, 18463, 19800, 21739, 23676, #La Liga 2
               13056, 16405, 17449, 18568, 19900, 22057, 23851, #Turkish Süper Lig
               13017, 16211, 17428, 18432, 19793, 21689, 23672 #Championship (2nd UK)
               
 ]
#Just considered leagues with at least 16 teams. 



# Loop through each interval and fetch data
for start, end in date_intervals:
    page = 1
    has_more = True # Flag for pagination
    file_path = f"fixtures_{start}_{end}.csv"  # Output file for each date range
    #first_page = True  # Track first page to write header only once
    fixtures_to_save = []

    while has_more:
        url = f"{BASE_URL}/between/{start}/{end}?api_token={API_KEY}&include=participants;lineups;events;formations;statistics;weatherReport&per_page=50&page={page}"
        print(f"Fetching data from {start} to {end}, page {page}")
        # print("We are in page:",page)
        response = requests.get(url)
        if response.status_code == 200:
            json_data = response.json()
            data = json_data.get("data", [])
            pagination = json_data.get("pagination")
            has_more = pagination.get("has_more", False)  # Check if more pages exist
            # print("HAs more: ", has_more )
            # print("data type:", type(has_more))
            # time.sleep(5)
            for fixture in data:
                if fixture.get("season_id") in season_list:  # Ensure correct key name "season_id"
                    team1 = fixture.get("participants", [{}])[0].get("name", "Unknown")
                    team1_location = fixture.get("participants", [{}])[0].get("meta", "Unknown").get("location")
                    # print(team1)
                    # print(team1_location)
                    team2 = fixture.get("participants", [{}])[1].get("name", "Unknown")
                    team2_location = fixture.get("participants", [{}])[1].get("meta", "Unknown").get("location")
                    if team1_location == "home":
                        home_team = team1
                        away_team = team2
                    elif team1_location == "away":
                        home_team = team2
                        away_team = team1
                    else:
                        raise NameError
                    
                    fixture_data = {
                        "fixture_id": fixture.get("id"),
                        "season_id": fixture.get("season_id"),
                        "round_id": fixture.get("round_id"),
                        "state_id": fixture.get("state_id"),
                        "date": fixture.get("starting_at"),
                        "home_team": home_team,
                        "away_team": away_team,
                        "participants": fixture.get("participants", []),
                        "lineups": fixture.get("lineups", []),
                        "formations": fixture.get("formations", []),
                        "statistics": fixture.get("statistics", []),
                        "events": fixture.get("events", []),
                        "weather": fixture.get("weatherreport", None)


                    }
                    fixtures_to_save.append(fixture_data)

            
            # Go to next page if needed
            if has_more:
                page += 1
        else:
            print(f"Failed to fetch data for {start} to {end}, page {page}. Status code: {response.status_code}")
            has_more = False  # Stop fetching if request fails
            
            
        # Convert to DataFrame and save to CSV (append mode)
    if fixtures_to_save:
        df = pd.DataFrame(fixtures_to_save)
        df.to_csv(file_path, mode='a', index=False, header=not os.path.exists(file_path))  # Write header if file doesn't exist
        #first_page = False  # After first page, don't write the header
            

print("Data fetching complete.")


Fetching data from 2018-07-01 to 2018-10-09, page 1
Fetching data from 2018-07-01 to 2018-10-09, page 2
Fetching data from 2018-07-01 to 2018-10-09, page 3
Fetching data from 2018-07-01 to 2018-10-09, page 4
Fetching data from 2018-07-01 to 2018-10-09, page 5
Fetching data from 2018-07-01 to 2018-10-09, page 6
Fetching data from 2018-07-01 to 2018-10-09, page 7
Fetching data from 2018-07-01 to 2018-10-09, page 8
Fetching data from 2018-07-01 to 2018-10-09, page 9
Fetching data from 2018-07-01 to 2018-10-09, page 10
Fetching data from 2018-07-01 to 2018-10-09, page 11
Fetching data from 2018-07-01 to 2018-10-09, page 12
Fetching data from 2018-07-01 to 2018-10-09, page 13
Fetching data from 2018-07-01 to 2018-10-09, page 14
Fetching data from 2018-07-01 to 2018-10-09, page 15
Fetching data from 2018-07-01 to 2018-10-09, page 16
Fetching data from 2018-07-01 to 2018-10-09, page 17
Fetching data from 2018-07-01 to 2018-10-09, page 18
Fetching data from 2018-07-01 to 2018-10-09, page 19
Fe

In [17]:
data_aux  = pd.read_csv("fixtures_2023-07-24_2023-11-01.csv")
data_aux.shape
data_aux.head()

,fixture_id,season_id,round_id,state_id,date,home_team,away_team,participants,lineups,formations,statistics,events,weather
0,18841807,21650,307093,5,2023-07-24 17:00:00,Korona Kielce,Śląsk Wrocław,"[{'id': 3962, 'sport_id': 1, 'country_id': 2, ...","[{'id': 3956080851, 'sport_id': 1, 'fixture_id...","[{'id': 522804, 'fixture_id': 18841807, 'parti...","[{'id': 22573233, 'fixture_id': 18841807, 'typ...","[{'id': 88779901, 'fixture_id': 18841807, 'per...","{'id': 258759, 'fixture_id': 18841807, 'venue_..."
1,18841819,21650,307094,5,2023-07-28 18:30:00,Ruch Chorzów,ŁKS Łódź,"[{'id': 12090, 'sport_id': 1, 'country_id': 2,...","[{'id': 3994026189, 'sport_id': 1, 'fixture_id...","[{'id': 523394, 'fixture_id': 18841819, 'parti...","[{'id': 22606566, 'fixture_id': 18841819, 'typ...","[{'id': 88858682, 'fixture_id': 18841819, 'per...","{'id': 261678, 'fixture_id': 18841819, 'venue_..."
2,18848035,21693,307685,5,2023-07-28 18:45:00,Union Saint-Gilloise,Anderlecht,"[{'id': 3958, 'sport_id': 1, 'country_id': 556...","[{'id': 3994410997, 'sport_id': 1, 'fixture_id...","[{'id': 523414, 'fixture_id': 18848035, 'parti...","[{'id': 22606802, 'fixture_id': 18848035, 'typ...","[{'id': 88859853, 'fixture_id': 18848035, 'per...","{'id': 259786, 'fixture_id': 18848035, 'venue_..."
3,18841821,21650,307094,5,2023-07-29 13:00:00,Warta Poznań,Górnik Zabrze,"[{'id': 3781, 'sport_id': 1, 'country_id': 2, ...","[{'id': 4008758958, 'sport_id': 1, 'fixture_id...","[{'id': 523596, 'fixture_id': 18841821, 'parti...","[{'id': 22621724, 'fixture_id': 18841821, 'typ...","[{'id': 88889822, 'fixture_id': 18841821, 'per...","{'id': 260307, 'fixture_id': 18841821, 'venue_..."
4,18848036,21693,307685,5,2023-07-29 14:00:00,AS Eupen,Westerlo,"[{'id': 2412, 'sport_id': 1, 'country_id': 556...","[{'id': 4011129261, 'sport_id': 1, 'fixture_id...","[{'id': 523652, 'fixture_id': 18848036, 'parti...","[{'id': 22627041, 'fixture_id': 18848036, 'typ...","[{'id': 88889320, 'fixture_id': 18848036, 'per...","{'id': 260340, 'fixture_id': 18848036, 'venue_..."


In [6]:
#open each csv file, sort them by date from older to newer (consider date is in the format "2022-02-13 17:00:00") and concatenate each file and return df_all_data
    
import pandas as pd
import os
from datetime import datetime

# raw_data_list = [
#     'fixtures_2018-08-01_2018-11-09.csv',
#     'fixtures_2018-11-10_2019-02-18.csv',
#     'fixtures_2019-02-19_2019-05-30.csv',
#     'fixtures_2019-05-31_2019-09-08.csv',
#     'fixtures_2019-09-09_2019-12-18.csv',
#     'fixtures_2019-12-19_2020-03-28.csv',
#     'fixtures_2020-03-29_2020-07-07.csv',
#     'fixtures_2020-07-08_2020-10-16.csv',
#     'fixtures_2020-10-17_2021-01-25.csv',
#     'fixtures_2021-01-26_2021-05-06.csv',
#     'fixtures_2021-05-07_2021-08-15.csv',
#     'fixtures_2021-08-16_2021-11-24.csv',
#     'fixtures_2021-11-25_2022-03-05.csv',
#     'fixtures_2022-03-06_2022-06-14.csv',
#     'fixtures_2022-06-15_2022-09-23.csv',
#     'fixtures_2022-09-24_2023-01-02.csv',
#     'fixtures_2023-01-03_2023-04-13.csv',
#     'fixtures_2023-04-14_2023-07-23.csv',
#     'fixtures_2023-07-24_2023-11-01.csv',
#     'fixtures_2023-11-02_2024-02-10.csv',
#     'fixtures_2024-02-11_2024-05-21.csv',
#     'fixtures_2024-05-22_2024-08-30.csv',
#     'fixtures_2024-08-31_2024-12-09.csv',
#     'fixtures_2024-12-10_2025-03-20.csv',
#     'fixtures_2025-03-21_2025-05-20.csv'
# ]

raw_data_list = [f"fixtures_{start}_{end}.csv" for start, end in date_intervals ]


def concatenate_sorted_csv(raw_data_list):
    
    all_data = []  # List to store individual DataFrames

    for file_path in raw_data_list:
        try:
            df = pd.read_csv(fr"./raw_data/{file_path}")
            df['date'] = pd.to_datetime(df['date'])
            df = df.sort_values(by='date')
            all_data.append(df)

        except FileNotFoundError:
            print(f"Error: File not found: {file_path}")
        except pd.errors.ParserError:
            print(f"Error: Could not parse CSV file: {file_path}")
        except KeyError:
            print(f"Error: 'date' column not found in {file_path}")
        except Exception as e:
            print(f"An unexpected error occurred with {file_path}: {e}")

    if all_data: #check if the list is not empty before concatenating
        df = pd.concat(all_data, ignore_index=True)
        return df
    

df_all_raw_data = concatenate_sorted_csv(raw_data_list)
df_all_raw_data.to_csv("df_all_raw_data.csv", index=False)


In [7]:
df_all_raw_data

,fixture_id,season_id,round_id,state_id,date,home_team,away_team,participants,lineups,formations,statistics,events,weather
0,10329241,12921,168236.0,5,2018-07-20 16:00:00,Miedź Legnica,Pogoń Szczecin,"[{'id': 244, 'sport_id': 1, 'country_id': 2, '...","[{'id': 5163775, 'sport_id': 1, 'fixture_id': ...","[{'id': 46276, 'fixture_id': 10329241, 'partic...","[{'id': 9939831, 'fixture_id': 10329241, 'type...","[{'id': 32401213, 'fixture_id': 10329241, 'per...",NaN
1,10329239,12921,168236.0,5,2018-07-20 18:30:00,Jagiellonia Białystok,Lechia Gdańsk,"[{'id': 890, 'sport_id': 1, 'country_id': 2, '...","[{'id': 5164574, 'sport_id': 1, 'fixture_id': ...","[{'id': 46272, 'fixture_id': 10329239, 'partic...","[{'id': 9765232, 'fixture_id': 10329239, 'type...","[{'id': 32401315, 'fixture_id': 10329239, 'per...",NaN
2,10329242,12921,168236.0,5,2018-07-21 13:30:00,Śląsk Wrocław,Cracovia Kraków,"[{'id': 3962, 'sport_id': 1, 'country_id': 2, ...","[{'id': 5164018, 'sport_id': 1, 'fixture_id': ...","[{'id': 46278, 'fixture_id': 10329242, 'partic...","[{'id': 9765908, 'fixture_id': 10329242, 'type...","[{'id': 32401294, 'fixture_id': 10329242, 'per...",NaN
3,10329243,12921,168236.0,5,2018-07-21 16:00:00,Wisła Kraków,Arka Gdynia,"[{'id': 331, 'sport_id': 1, 'country_id': 2, '...","[{'id': 5163837, 'sport_id': 1, 'fixture_id': ...","[{'id': 46280, 'fixture_id': 10329243, 'partic...","[{'id': 9765341, 'fixture_id': 10329243, 'type...","[{'id': 32401282, 'fixture_id': 10329243, 'per...",NaN
4,10329240,12921,168236.0,5,2018-07-21 18:30:00,Legia Warszawa,Zagłębie Lubin,"[{'id': 349, 'sport_id': 1, 'country_id': 2, '...","[{'id': 5164094, 'sport_id': 1, 'fixture_id': ...","[{'id': 46274, 'fixture_id': 10329240, 'partic...","[{'id': 9764566, 'fixture_id': 10329240, 'type...","[{'id': 32401512, 'fixture_id': 10329240, 'per...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33362,19145090,23676,340319.0,1,2025-05-19 18:30:00,Córdoba,Mirandés,"[{'id': 931, 'sport_id': 1, 'country_id': 32, ...",[],[],[],[],NaN
33363,19376848,23839,NaN,1,2025-05-19 18:30:00,17th ranked,16th ranked,"[{'id': 267744, 'sport_id': 1, 'country_id': 1...",[],[],[],[],NaN
33364,19135032,23614,339272.0,1,2025-05-19 19:00:00,Brighton & Hove Albion,Liverpool,"[{'id': 78, 'sport_id': 1, 'country_id': 462, ...",[],[],[],[],NaN
33365,19135034,23614,339272.0,1,2025-05-20 19:00:00,Crystal Palace,Wolverhampton Wanderers,"[{'id': 51, 'sport_id': 1, 'country_id': 462, ...",[],[],[],[],NaN


## Getting the round name

In [8]:
import requests
import pandas as pd
import os
import time

BASE_URL = "https://api.sportmonks.com/v3/football/rounds/seasons"




# Loop through each interval and fetch data
for season_aux in season_list:
    page = 1
    has_more = True # Flag for pagination
    file_path = f"season_{season_aux}_rounds.csv"  # Output file for each date range
    rounds_to_save = []

    while has_more:
        url = f"{BASE_URL}/{season_aux}?api_token={API_KEY}&per_page=50&page={page}"
        print(f"Fetching data from {season_aux}, page {page}")
        # print("We are in page:",page)
        response = requests.get(url)
        if response.status_code == 200:
            json_data = response.json()
            data = json_data.get("data", [])
            has_more = pagination.get("has_more", False) if pagination else False
            # print("Has more: ", has_more )
            # print("data type:", type(has_more))
            for round in data:
                round_data = {
                    "round_id": round.get("id"),
                    "round_name": round.get("name"),
                }
                rounds_to_save.append(round_data)
            
            # Go to next page if needed
            if has_more:
                page += 1
        else:
            print(f"Failed to fetch data for season {season_aux}, page {page}. Status code: {response.status_code}")
            has_more = False  # Stop fetching if request fails
            
            
        # Convert to DataFrame and save to CSV (append mode)
    if rounds_to_save:
        df = pd.DataFrame(rounds_to_save)
        df.to_csv(file_path, mode='a', index=False, header=not os.path.exists(file_path))  # Write header if file doesn't exist
            

print("Data fetching complete.")


Fetching data from 12962, page 1
Fetching data from 16036, page 1
Fetching data from 17420, page 1
Fetching data from 18378, page 1
Fetching data from 19734, page 1
Fetching data from 21646, page 1


KeyboardInterrupt: 

In [9]:
#open each csv file, sort them by date from older to newer (consider date is in the format "2022-02-13 17:00:00") and concatenate each file and return df_all_data
    
import pandas as pd
import os
from datetime import datetime

rounds_path = [f'season_{season}_rounds.csv' for season in season_list]




def concatenate_sorted_csv(rounds_path):
    
    df_rounds = []  # List to store individual DataFrames

    for file_path in rounds_path:
        try:
            df = pd.read_csv(fr"./raw_data/{file_path}")
            df_rounds.append(df)

        except FileNotFoundError:
            print(f"Error: File not found: {file_path}")
        except pd.errors.ParserError:
            print(f"Error: Could not parse CSV file: {file_path}")
        except KeyError:
            print(f"Error: 'date' column not found in {file_path}")
        except Exception as e:
            print(f"An unexpected error occurred with {file_path}: {e}")

    if df_rounds: #check if the list is not empty before concatenating
        df = pd.concat(df_rounds, ignore_index=True)
        return df
    

df_rounds = concatenate_sorted_csv(rounds_path)
df_rounds.to_csv("df_rounds.csv", index=False)


season_id
17428        557
13017        557
21689        557
19793        557
18432        557
            ... 
16010        296
12921        296
12951        270
17287        240
16276        240
Name: count, Length: 91, dtype: int64
